In [ ]:
import requests

In [ ]:
res = requests.get('https://arclim.mma.gob.cl/api/capas')
if res.status_code == 200:
    data = res.json()
    data
else:
    print(f'Error: {res.status_code}')

In [ ]:
data

In [ ]:
res = requests.get('https://arclim.mma.gob.cl/api/attributos/arclim_raster_5km/')
if res.status_code == 200:
    data = res.json()

else:
    print(f'Error: {res.status_code}')

In [ ]:
data

In [ ]:
res = requests.get('https://arclim.mma.gob.cl/api/datos/arclim_raster_5km/json/?attributes=id,$CLIMA$hot_days$annual$present')
if res.status_code == 200:
    data = res.json()

else:
    print(f'Error: {res.status_code}')

In [ ]:
data

### local database

In [ ]:
import zipfile

In [ ]:
import os
import zipfile

def decompress_zip(zip_path, extract_to):
    """
    Decompresses a single ZIP file.
    
    Args:
    zip_path (str): The path to the ZIP file.
    extract_to (str): The directory to extract files to.
    """
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
        print(f'Extracted {zip_path} to {extract_to}')

# Define the directory containing the ZIP files and the directory to extract to
zip_files_directory = '/mnt/d/arclim/IndicesClimaticosARCLIM/'
extracted_files_directory = '/mnt/d/arclim/IndicesClimaticosARCLIM/'

# Ensure the extraction directory exists
os.makedirs(extracted_files_directory, exist_ok=True)

# Iterate over each ZIP file in the directory and decompress it
for filename in os.listdir(zip_files_directory):
    if filename.endswith('.zip'):
        zip_path = os.path.join(zip_files_directory, filename)
        decompress_zip(zip_path, extracted_files_directory)


In [ ]:
import xarray as xr
import rioxarray as rioxr
import requests
from tqdm import tqdm
import os
import zipfile
import pandas as pd


class arclimdata:
    """
    A class to manage downloading, extracting, and loading climate data from the Arclim database.

    Attributes:
    working_dir (str): The directory where the data will be downloaded and extracted.
    climate_indices (pd.DataFrame): DataFrame containing climate indices information.
    """

    def __init__(self, working_dir):
        """
        Initializes the ArclimData object with a working directory and loads the climate indices.

        Args:
        working_dir (str): The directory where the data will be downloaded and extracted.
        """
        self.working_dir = working_dir
        self.climate_indices = pd.read_csv(f"{working_dir}arclim_climate_indices.csv")

    def decompress_zip(self, zip_path, extract_to):
        """
        Decompresses a single ZIP file.

        Args:
        zip_path (str): The path to the ZIP file.
        extract_to (str): The directory to extract files to.

        Returns:
        None
        """
        with zipfile.ZipFile(zip_path, "r") as zip_ref:
            zip_ref.extractall(extract_to)
            print(f"Extracted {zip_path} to {extract_to}")

    def download_climate_data(self):
        """
        Downloads and extracts the Arclim climate data ZIP file to the working directory.

        Returns:
        None
        """

        url = "https://arclim.mma.gob.cl/media/ClimateIndex/geotiff/IndicesClimaticosARCLIM.zip"  # Replace with your file URL
        local_filename = f"{self.working_dir}/IndicesClimaticosARCLIM.zip"  # Replace with your desired local file name
        print(f"Downloading Arclim Climate Database")
        with requests.get(url, stream=True) as response:
            response.raise_for_status()
            total_size = int(response.headers.get("content-length", 0))
            block_size = 8192

            with open(local_filename, "wb") as file, tqdm(
                total=total_size, unit="iB", unit_scale=True
            ) as progress_bar:
                for chunk in response.iter_content(chunk_size=block_size):
                    file.write(chunk)
                    progress_bar.update(len(chunk))

        print(f"Downloaded {local_filename}")

        print("Extracting files")
        zip_files_directory = f"{self.working_dir}/IndicesClimaticosARCLIM/"
        extracted_files_directory = f"{self.working_dir}/IndicesClimaticosARCLIM/"

        # Ensure the extraction directory exists
        os.makedirs(extracted_files_directory, exist_ok=True)
        self.decompress_zip(
            f"{folder}/IndicesClimaticosARCLIM.zip",
            f"{folder}/IndicesClimaticosARCLIM/",
        )
        # Define the directory containing the ZIP files and the directory to extract to

        # Iterate over each ZIP file in the directory and decompress it
        for filename in os.listdir(zip_files_directory):
            if filename.endswith(".zip"):
                zip_path = os.path.join(zip_files_directory, filename)
                self.decompress_zip(zip_path, extracted_files_directory)

    def load_climate_indice(self, name, period, months, format):
        """
        Loads a specific climate index as an xarray object.

        Args:
        name (str): The name of the climate index, must be one of the "Código" in self.climate_indices.
        period (str): The period of the data, must be one of 'present' (1971-2010), 'future' (2035-2065), or 'delta' (difference between future and present).
        months (str): The months or period of interest, must be one of 'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'ago', 'sep', 'oct', 'nov', 'dec', 'djf', 'mam', 'jja', 'son', 'summer', 'winter', 'annual'.
        format (str): The format of the returned object, currently supports 'xarray'.

        Returns:
        xarray.DataArray: The loaded climate index data as an xarray object.

        Raises:
        AssertionError: If the name, period, or months arguments are not valid.

        """
        if format == "xarray":

            assert name in list(
                self.climate_indices["Código"]
            ), f"{name} is not of ARCLIM Climate Indices. Check self.climate_indices DataFrame."
            assert period in [
                "future",
                "present",
                "delta",
            ], f"period should be one of 'future', 'present' or 'delta'"
            assert months in [
                "jan",
                "feb",
                "mar",
                "apr",
                "may",
                "jan",
                "jul",
                "ago",
                "sep",
                "oct",
                "nov",
                "dec",
                "djf",
                "mam",
                "jja",
                "son",
                "summer",
                "winter",
                "annual",
            ], f"months should be one of 'jan', 'feb', 'mar', 'apr', 'may', 'jan', 'jul', 'ago', 'sep', 'oct', 'nov', 'dec', 'djf', 'mam', 'jja', 'son', 'summer', 'winter' or 'annual'"

            ds = rioxr.open_rasterio(
                f"{self.working_dir}/IndicesClimaticosARCLIM/{name}/{name}_{period}_{months}_latlon.tif"
            )

            return ds

In [1]:
import os
os.chdir("/mnt/d/arclim/") # change dir to where arclimdata.py is located
from arclimdata import arclimdata
folder = "/mnt/d/arclim/" # where arclim data will be stored
arclim = arclimdata(working_dir = folder)
# arclim.download_climate_data() # download and extract arclim climate indices geotifs
arclim.climate_indices ## returns a pandas dataframe with information about climate indices
arclim.load_climate_indice(name = 'consecutive_days_over_25C', period = 'future', months = 'annual', format = 'xarray') # return a specific climate index as an xarray object.


,Categoría,Código,Nombre,Descripción,Unidades,∆
0,Calor,consecutive_days_over_25C,Olas de calor > 25ºC,Número de días que la temperatura máxima diari...,dias,abs
1,Calor,consecutive_days_over_28C,Olas de calor > 28ºC,Número de días que la temperatura máxima diari...,dias,abs
2,Calor,consecutive_days_over_30C,Olas de calor > 30ºC,Número de días que la temperatura máxima diari...,dias,abs
3,Calor,daily_temperature_range,Amplitud térmica,Valor medio de la amplitud térmica diaria (Tma...,grados,abs
4,Calor,degdays_above_15C,Grados-dia sobre 15ºC,Grados-dia sobre 15ºC,grados-dias,abs
...,...,...,...,...,...,...
60,Humedad,huss_mean,Humedad específica,Valor medio de la humedad específica (g/Kg),g/kg,rel
61,Insolación,rsds_max,Insolación solar máxima diaria,Valor máximo de la irradiación global horizont...,W/m2,abs
62,Insolación,rsds_mean,Insolación solar diaria,Valor medio de la irradiación global horizontal,W/m2,abs
63,Insolación,rsds_min,Insolación solar mínima diaria,Valor mínimo de la irradiación global horizont...,W/m2,abs


In [ ]:
arclim.load_climate_indice(name = 'consecutive_days_over_25C', period = 'future', months = 'annual', format = 'xarray')

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup

html_content = """
<table class="uk-table uk-table-small uk-table-striped">
    <thead>
        <tr>
            <th> Categoría </th> 
            <th> Código </th> 
            <th> Nombre </th> 
            <th> Descripción </th> 
            <th> Unidades </th> 
            <th> ∆ </th> 
        </tr>
    </thead>
    <tbody>
        <tr>
            <td> Calor </td> 
            <td> consecutive_days_over_25C </td> 
            <td> Olas de calor &gt; 25ºC </td>
            <td> Número de días que la temperatura máxima diaria supera 25 ºC durante 3 o más días seguidos </td>
            <td> dias </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Calor </td> 
            <td> consecutive_days_over_28C </td> 
            <td> Olas de calor &gt; 28ºC </td>
            <td> Número de días que la temperatura máxima diaria supera 28 ºC durante 3 o más días seguidos </td>
            <td> dias </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Calor </td> 
            <td> consecutive_days_over_30C </td> 
            <td> Olas de calor &gt; 30ºC </td>
            <td> Número de días que la temperatura máxima diaria supera 30 ºC durante 3 o más días seguidos </td>
            <td> dias </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Calor </td> 
            <td> daily_temperature_range </td> 
            <td> Amplitud térmica </td>
            <td> Valor medio de la amplitud térmica diaria (Tmax - Tmin). </td>
            <td> grados </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Calor </td> 
            <td> degdays_above_15C </td> 
            <td> Grados-dia sobre 15ºC </td>
            <td> Grados-dia sobre 15ºC </td>
            <td> grados-dias </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Calor </td> 
            <td> degdays_above_20C </td> 
            <td> Grados-dia sobre 20ºC </td>
            <td> Grados-dia sobre 20ºC </td>
            <td> grados-dias </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Calor </td> 
            <td> degdays_above_25 </td> 
            <td> Grados-dia sobre 25ºC </td>
            <td> Grados-dia sobre 25ºC </td>
            <td> grados-dias </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Calor </td> 
            <td> degdays_above_30 </td> 
            <td> Grados-dia sobre 30ºC </td>
            <td> Grados-dia sobre 30ºC </td>
            <td> grados-dias </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Calor </td> 
            <td> hot_days </td> 
            <td> Dias calurosos (&gt;30ºC) </td>
            <td> Número de días en que la temperatura máxima supera 30ºC. </td>
            <td> dias </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Calor </td> 
            <td> hottest_day </td> 
            <td> Día más cálido </td>
            <td> Temperatura (en ºC) del día más cálido del periodo considerado (año, estación, mes) </td>
            <td> ºC </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Calor </td> 
            <td> mean_temperature </td> 
            <td> Temperatura media </td>
            <td> Valor medio de la temperatura media diaria, calculado como 0.5*(Tmax + Tmin) </td>
            <td> grados </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Calor </td> 
            <td> quite_hot_days </td> 
            <td> Dias relativamente cálidos (&gt; 28ºC) </td>
            <td> Número de días en que la temperatura máxima supera 28ºC. </td>
            <td> dias </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Calor </td> 
            <td> summer_days </td> 
            <td> Días de verano (&gt; 25ºC) </td>
            <td> Número de días en que la temperatura supera 25ºC </td>
            <td> dias </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Calor </td> 
            <td> tasmax_mean </td> 
            <td> Promedio de la temperatura máxima diaria </td>
            <td> Promedio de la temperatura máxima diaria </td>
            <td> ºC </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Calor </td> 
            <td> tropical_nights </td> 
            <td> Noches tropicales </td>
            <td> Número de días en que la temperatura mínima supera 20ºC </td>
            <td> dias </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Calor </td> 
            <td> very_hot_days </td> 
            <td> Dias muy cálidos </td>
            <td> Número de días en que la temperatura supera 34ºC </td>
            <td> dias </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Calor </td> 
            <td> warm_days </td> 
            <td> Días cálidos </td>
            <td> Número de días en que la temperatura supera el umbral definido como el percentil 90 de la temperatura diaria en el periodo de referencia (1980 a 2010). </td>
            <td> dias </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Calor </td> 
            <td> warmest_night </td> 
            <td> Noche más cálida </td>
            <td> Temperatura mínima en el día de mayor temperatura mínima durante el periodo considerado (año, estación, mes) </td>
            <td> ºC </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Calor </td> 
            <td> warm_nights </td> 
            <td> Noches cálidas </td>
            <td> Número de días en que la temperatura mínima supera el umbral definido como el percentil 90 de la temperatura mínima diaria en el periodo de referencia (1980 a 2010). </td>
            <td> dias </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Calor </td> 
            <td> warm_nights_duration_index_long </td> 
            <td> Duración de periodos de noches cálidas (&gt; 6 dias) </td>
            <td> Numero de días consecutivos (en cadenas de al menos 6) que la temperatura mínima supera el percentil 90. </td>
            <td> dias </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Calor </td> 
            <td> warm_nights_duration_index_short </td> 
            <td> Duración de periodos de noches cálidas (&gt; 3 dias) </td>
            <td> Numero de días consecutivos (en cadenas de al menos 3) en que la temperatura mínima supera el percentil 90. </td>
            <td> dias </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Calor </td> 
            <td> warm_spell_duration_index_long </td> 
            <td> Duración de episodios cálidos (&gt; 6 dias) </td>
            <td> Numero de dias con al menos 6 dias consecutivos en que la temperatura máxima supera el percentil 90. </td>
            <td> dias </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Calor </td> 
            <td> warm_spell_duration_index_short </td> 
            <td> Duración de episodios cálidos (&gt; 3 dias) </td>
            <td> Numero de dias con al menos 3 dias consecutivos en que la temperatura máxima supera el percentil 90. </td>
            <td> dias </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Frío </td> 
            <td> coldest_day </td> 
            <td> Día más frío </td>
            <td> Temperatura máxima del día más frió del periodo considerado (año, estación, mes) </td>
            <td> ºC </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Frío </td> 
            <td> coldest_night </td> 
            <td> Noche más fría </td>
            <td> Temperatura de la noche más fría (temperatura mínima) del periodo considerado (año, estación, mes) </td>
            <td> ºC </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Frío </td> 
            <td> cold_spell_duration_index_long </td> 
            <td> Duración de episodios fríos (&gt; 6 días) </td>
            <td> Numero de días con al menos 6 días consecutivos en que la temperatura mínima no supera el percentíl 10 de la temperatura mínima diaria en el periodo de referencia (1980 a 2010). </td>
            <td> dias </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Frío </td> 
            <td> cold_spell_duration_index_short </td> 
            <td> Duración de episodios fríos (&gt; 3 días) </td>
            <td> Numero de días con al menos 6 días consecutivos en que la temperatura mínima no supera el percentíl 10 de la temperatura mínima diaria en el periodo de referencia (1980 a 2010). </td>
            <td> dias </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Frío </td> 
            <td> cool_days </td> 
            <td> Días fríos </td>
            <td> Numero de días en que la temperatura máxima en menor que el percentíl 10. </td>
            <td> dias </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Frío </td> 
            <td> cool_nights </td> 
            <td> Noches frías </td>
            <td> Numero de días en que la temperatura mínima en menor que el percentíl 10. </td>
            <td> dias </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Frío </td> 
            <td> degdays_below_0c </td> 
            <td> Grados-dia bajo 0ºC </td>
            <td> Grados-dia bajo 0ºC </td>
            <td> grados-dias </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Frío </td> 
            <td> degdays_below_10C </td> 
            <td> Grados-dia bajo 10ºC </td>
            <td> Grados-dia bajo 10ºC </td>
            <td> grados-dias </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Frío </td> 
            <td> degdays_below_15C </td> 
            <td> Grados-dia bajo 15ºC </td>
            <td> Grados-dia bajo 15ºC </td>
            <td> grados-dias </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Frío </td> 
            <td> degdays_below_5C </td> 
            <td> Grados-dia bajo 5ºC </td>
            <td> Grados-dia bajo 5ºC </td>
            <td> grados-dias </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Frío </td> 
            <td> frost_days </td> 
            <td> Días de  escarcha </td>
            <td> Número de días en que la temperatura mínima es menor que 0ºC </td>
            <td> dias </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Frío </td> 
            <td> ice_days </td> 
            <td> Días de hielo </td>
            <td> Número de días en que la temperatura máxima es menor que 0ºC </td>
            <td> dias </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Frío </td> 
            <td> tasmin_mean </td> 
            <td> Promedio de la temperatura mínima diaria </td>
            <td> Promedio de la temperatura mínima diaria </td>
            <td> ºC </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Precipitación </td> 
            <td> consecutive_dry_days </td> 
            <td> Días secos consecutivos </td>
            <td> Número máximo de días consecutivos en que la precipitación diaria no supera 1 mm </td>
            <td> dias </td>
            <td> rel </td>
        </tr>
        <tr>
            <td> Precipitación </td> 
            <td> consecutive_wet_days </td> 
            <td> Días húmedos consecutivos </td>
            <td> Número máximo de días consecutivos en que la precipitación diaria supera 1 mm </td>
            <td> dias </td>
            <td> rel </td>
        </tr>
        <tr>
            <td> Precipitación </td> 
            <td> drought_periods </td> 
            <td> Frecuencia de sequía </td>
            <td> Frecuencia de periodos en que la precipitación acumulada es menor al 75% del promedio la precipitación acumulada en el periodo de referencia (1980 a 2010) </td>
            <td> % </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Precipitación </td> 
            <td> eto_mean </td> 
            <td> Evaportraspiración Potencial </td>
            <td> Evaportraspiración Potencial medio, calculado usado el metodo de Penman-Montieth (FAO56) </td>
            <td> mm </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Precipitación </td> 
            <td> maximum_daily_precipitation </td> 
            <td> Precipitación máxima diaria </td>
            <td> Precipitación máxima diaria </td>
            <td> dias </td>
            <td> rel </td>
        </tr>
        <tr>
            <td> Precipitación </td> 
            <td> number_of_heavy_precipitation_days </td> 
            <td> Días de precipitación intensa </td>
            <td> Número de días en que la precipitación diaria supera 10 mm </td>
            <td> dias </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Precipitación </td> 
            <td> number_of_very_heavy_precipitation_days </td> 
            <td> Días de precipitación muy intensa </td>
            <td> Número de días en que la precipitación diaria supera 20 mm </td>
            <td> dias </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Precipitación </td> 
            <td> pr_max </td> 
            <td> Precipitación máxima diaria </td>
            <td> Valor máximo de la precipitación (lluvia y nieve) diaria </td>
            <td> mm </td>
            <td> rel </td>
        </tr>
        <tr>
            <td> Precipitación </td> 
            <td> pr_sum </td> 
            <td> Precipitación acumulada </td>
            <td> Cantidad de precipitación (lluvia y nieve) acumulada </td>
            <td> mm </td>
            <td> rel </td>
        </tr>
        <tr>
            <td> Precipitación </td> 
            <td> simple_precip_intensity_index </td> 
            <td> Indice simple de intensidad de precipitación </td>
            <td> Promedio de la precipitación diaria considerando sólo los días en que la precipitación supera 1mm/día </td>
            <td> mm/día </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Precipitación </td> 
            <td> spi </td> 
            <td> Standadized Precipitation Index </td>
            <td> Standadized Precipitation Index </td>
            <td> Z </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Precipitación </td> 
            <td> total_from_extremely_wet_days </td> 
            <td> Acumulación en días muy húmedos </td>
            <td> Precipitación acumulada en los días en que la precipitación diaria supera el percentil 95  del periodo de referencia (1980 a 2010) </td>
            <td> mm </td>
            <td> rel </td>
        </tr>
        <tr>
            <td> Precipitación </td> 
            <td> total_from_very_wet_days </td> 
            <td> Acumulación en días húmedos </td>
            <td> Precipitación acumulada en los días en que la precipitación diaria supera el percentil 99  del periodo de referencia (1980 a 2010) </td>
            <td> mm </td>
            <td> rel </td>
        </tr>
        <tr>
            <td> Lluvia </td> 
            <td> rain_max </td> 
            <td> Lluvia máxima diaria </td>
            <td> Valor máximo de la lluvia (agua liquida) acumulada en un día </td>
            <td> mm </td>
            <td> rel </td>
        </tr>
        <tr>
            <td> Lluvia </td> 
            <td> rain_sum </td> 
            <td> Lluvia acumulada </td>
            <td> Cantidad de lluvia (agua líquida) acumulada en milímetros durante el período (año, estación, o mes) </td>
            <td> mm </td>
            <td> rel </td>
        </tr>
        <tr>
            <td> Nieve </td> 
            <td> snow_max </td> 
            <td> Nieve máxima diaria </td>
            <td> Valor máximo de la nieve acumulada en un día,  expresada en mm de agua equivalente </td>
            <td> mm </td>
            <td> rel </td>
        </tr>
        <tr>
            <td> Nieve </td> 
            <td> snow_sum </td> 
            <td> Nieve acumulada </td>
            <td> Cantidad de nieve acumulada, expresada en mm de agua equivalente </td>
            <td> mm </td>
            <td> rel </td>
        </tr>
        <tr>
            <td> Viento </td> 
            <td> uas_mean </td> 
            <td> Viento zonal medio </td>
            <td> Valor medio del componente zonal (este-oeste) del viento </td>
            <td> m/s </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Viento </td> 
            <td> vas_mean </td> 
            <td> Viento meridional medio </td>
            <td> Valor medio del componente meridional (norte-sur) del viento </td>
            <td> m/s </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Viento </td> 
            <td> vel_max </td> 
            <td> Viento máximo diario </td>
            <td> Valor medio del máximo diario de la magnitud del viento </td>
            <td> m/s </td>
            <td> rel </td>
        </tr>
        <tr>
            <td> Viento </td> 
            <td> vel_mean </td> 
            <td> Viento medio </td>
            <td> Valor medio de la magnitud del viento </td>
            <td> m/s </td>
            <td> rel </td>
        </tr>
        <tr>
            <td> Humedad </td> 
            <td> hursmax_mean </td> 
            <td> Humedad relativa máxima diaria </td>
            <td> Valor medio de la humedad relativa (%) </td>
            <td> % </td>
            <td> rel </td>
        </tr>
        <tr>
            <td> Humedad </td> 
            <td> hurs_mean </td> 
            <td> Humedad relativa media diaria </td>
            <td> Valor medio del máximo diario de la humedad relativa (%) </td>
            <td> % </td>
            <td> rel </td>
        </tr>
        <tr>
            <td> Humedad </td> 
            <td> hursmin_mean </td> 
            <td> Humedad relativa mínima diaria </td>
            <td> Valor medio del mínimo diario de la humedad relativa (%) </td>
            <td> % </td>
            <td> rel </td>
        </tr>
        <tr>
            <td> Humedad </td> 
            <td> huss_mean </td> 
            <td> Humedad específica </td>
            <td> Valor medio de la humedad específica (g/Kg) </td>
            <td> g/kg </td>
            <td> rel </td>
        </tr>
        <tr>
            <td> Insolación </td> 
            <td> rsds_max </td> 
            <td> Insolación solar máxima diaria </td>
            <td> Valor máximo de la irradiación global horizontal diaria (W/m2) </td>
            <td> W/m2 </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Insolación </td> 
            <td> rsds_mean </td> 
            <td> Insolación solar diaria </td>
            <td> Valor medio de la irradiación global horizontal </td>
            <td> W/m2 </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Insolación </td> 
            <td> rsds_min </td> 
            <td> Insolación solar mínima diaria </td>
            <td> Valor mínimo de la irradiación global horizontal diaria </td>
            <td> W/m2 </td>
            <td> abs </td>
        </tr>
        <tr>
            <td> Presión </td> 
            <td> ps_mean </td> 
            <td> Presión atmosférica media </td>
            <td> Valor medio de la presión atmosférica en superficie </td>
            <td> hPa </td>
            <td> abs </td>
        </tr>
    </tbody>
</table>
"""

# Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Find the table
table = soup.find('table')

# Extract headers
headers = [header.get_text(strip=True) for header in table.find_all('th')]

# Extract rows
rows = []
for row in table.find_all('tr'):
    cells = row.find_all('td')
    if len(cells) > 0:
        rows.append([cell.get_text(strip=True) for cell in cells])

# Create a DataFrame
df = pd.DataFrame(rows, columns=headers)

# Save to CSV
df.to_csv('/mnt/d/arclim/arclim_climate_indices.csv', index=False)

print("CSV file has been created.")
